In [6]:
'''
From Keras examples:
https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py

Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

In [7]:
# Download the corpus
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 600901


In [8]:
# Variable text is a string.
# set(text) gets the list of all characters occurring here (26 letters, plus any other symbols)
chars = sorted(list(set(text)))
print('total chars:', len(chars))

total chars: 59


In [9]:
# Create two dictionaries to make it easy to look up the index of a character or the character for an index
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [10]:
# Now scroll through the corpus, (step) characters at a time, 
# and chop up the original text into sequences of (maxlen) characters.
maxlen = 20
step = 20

sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 30045


In [11]:
"""
This is important:

Here we vectorize the text representation...
Turn it into X and Y vectors that a neural network can handle.

X is a matrix: 
* number of rows equal to number of 40-character chunks
* number of columns equal to number of characters in the 40 character chunk
"""

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [12]:
# Construct the neurla network:
# Long-Short Term Memory sequential neural network
print('Build model...')
model = Sequential()
model.add(LSTM(64, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

Build model...


In [13]:
# Set the optimizer that will actually solve for the neural network weights
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer)

In [14]:
"""
Helper function: sample()

Given an experiment performed multiple times,
with a fixed set of possible outcomes (preds),
we can use a multinomial distribution to quantify 
the probability of a certain set of outcomes.

Sampling the experimental outcomes `n` times in `n` experiments 
will quantify the probability of an outcome `X_i = [X_0, X_1, ..., X_p]`
representing  the number of times the outcome was `i`.

This samples the multinomial distribution randomly,
and returns the most likely outcome.
"""

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [15]:
"""
Training the Model:


"""



# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=64, nb_epoch=1, verbose=0)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
        sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1

----- diversity: 0.2
----- Generating with seed: "utwear
     your sou"
utwear
     your sous the aresting of the henting the arestere of the sopreresting and and and and and and and and of the anded the hating the hating the the the and the hing the seration of the senting the asting and and the the serestare the hating the serenting and and and and and and and and and and and and and as and in the hating and and the hation the hating the torther and and the the hat of the hating the ar

----- diversity: 0.5
----- Generating with seed: "utwear
     your sou"
utwear
     your sous ald the tint whe hatter intenes and that anderessed and is never the installes on in the ong aresurning as thith the sesting the alded wat monger as are the sost as pration of that and act the sinting the rallion and and and atherscantion mare thith whe hich as the reveane wat ard and the hating whing a the secteratere somast and preraling of pr


----- diversity: 0.2
----- Generating with seed: "lation to opposing m"
lation to opposing may the would and the some of the reast the some of the every a the still of the reastion, and the so masie the pertance of the evilicated and the in the some of the revilicated and the evile of the reast for the some of the self the evile in the reverent of a such a propouse and the in the resting the spection, a the interself and the some of the has a serple and the same in the world and the some

----- diversity: 0.5
----- Generating with seed: "lation to opposing m"
lation to opposing morality of the
cause in the has of the reas far it in in the reast prowals and agrent a the mastered of the gest who as the still and maries, and deese of the elostire for the evile in the has who rest and presenter which the revene the moral conseration, and so main of repentione of the
some of the consententing whech a a rection of the son the parse to the consent like a muatible and agard must 

----- diver

g a relatively briefore in the in the most and the which and every one which and every for the in the in the self the perion of the in the self the self sech and again the in the in the which a profent the who ar all the most and proces of the german for the some of the in the in the in the consedver of the conself is the would the moder the sempire the in the in the most and deling the freat of the inely one the in 

----- diversity: 0.5
----- Generating with seed: "g a relatively brief"
g a relatively brief one uncertim to the openifes, and consedver of man in the strighd and agenture the himself and person in the self and strangual howak in the
periop in the in the which is they higher of the some of the moderr and who der in the most itself degeres of the firement for the not that in the perself in sperionary as as its himal higher the would and the man speenes and the one repose be still of the d

----- diversity: 1.0
----- Generating with seed: "g a relatively brief"
g a relative

 of the eyes must the world in the most and the inteller with the is the german soul sure of the intention. it is the some and stall the german the intellect and spection of the such a sury a such a such a such as among of the world of the general consciemate of the german with a feeling the too and the some and again the inteller of in the some and again of the is or and again to the is a sure of a proporden of the 

----- diversity: 0.5
----- Generating with seed: " of the eyes must th"
 of the eyes must the intellect of a with the propless and the intist and so fan and without the superfin of in the in the strone of the proplet to and not as any and manifes and the the sign of himself the intelless and frener of the some under, actions of the some himself--and it is still world of the inteller and germanive and con the masitally in the may be a the as and master, and the signien of the perplan with

----- diversity: 1.0
----- Generating with seed: " of the eyes must th"
 of the eyes

ven in the humanest the some to the such a soul old and still of the reals the so nature and still suffict of the concestion of the some to contit to the such a sees to the sure to the strous of the such a sees to the would the some himself and the some to the sure the intest and manling to the strong the some the intercalion to the strous who dient of the such a see in the may at the such a see in the such a serut o

----- diversity: 0.5
----- Generating with seed: "ven in the humanest "
ven in the humanest and deartion of the suffining to prenaile of the reason the morality for the reasing to the wastion of regarficion do the indiling of the inupilies of all the intord the entif belief and present it is proser is and still mann mist of the reast has hels and not the still of the such as some in the pnotoration to an as his as an as he self says, no str and antith eleves it is itself the german sing 

----- diversity: 1.0
----- Generating with seed: "ven in the humanest "
ven in the h

...

ortably couching sould the love himself, and every have been in the reason of the reason and even the world, who are that is the some that what however, and so farper in the sures and evistard of the sures and so man as a stronger in the sures and evistrairs, that his
orrans, the some in the sures a soul himself, and even the some in the priness, the some that a soul himself, and even the self, and even his secration

----- diversity: 0.5
----- Generating with seed: "ortably couching sou"
ortably couching soul-liging,wkand as a soul reliever in the roporal to the individualion deepre is the some that has as as a senting and has procession, and
are his own limes allity, consict and he is in the reason which these this sould the some soctainces nown the world and even the same it is and which is a serves and develing of the moraled and more and even through in frigne, this reliever understand as to any


----- diversity: 1.0
----- Generating with seed: "ortably couching sou"
ortably couc

the such a not the something of the indering that the so man and still and calles and still of the germanilable that the sensament of such a never, the such a devilingly and evil of manification and profound of the something of the such a many possed to a sering that the inders of the peringbering to the something of the world of the peringual himself as a the sensame of a creating the peringual of 

----- diversity: 0.5
----- Generating with seed: "igest and "change
th"
igest and "change
the physe more a many post and possed of phy man dif nemedually and side of the princes howr a call, which comerances and evilifities and such a man german periopiritalingty, and not any perhaps a per a made that the relining of the speality of the pagenes him a scholard of itself regard to a came all thinks at which a periops amost anewall as have protentime the animachy, and long. the tran and the

----- diversity: 1.0
----- Generating with seed: "igest and "change
th"
igest and "change
the
madely e

g, to lurk, to hark!en and still and still of the present and so man and so manifations and still offere the morality of the such a self-danduature and to the soull the so man and so man and the most and still oftin the morality, as a surn to the world to be the stronges to the so man and so man and so man and sutseavere of the sould the inding of the morality of the most and still of the most may a so man in the sec

----- diversity: 0.5
----- Generating with seed: "g, to lurk, to hark!"
g, to lurk, to hark!en and morals--becamally beligion too
clation and prooftent and consediaise that so moders and still in his offerent they are in the most see instinct of the sed, the morality of classianable the sexuted suction, and as his own perion
and a veritent, the and in the sturled the woman and stapicied to the intreally profoundening elientary of the world to plact happers of the morality; dortion of the

----- diversity: 1.0
----- Generating with seed: "g, to lurk, to hark!"
g, to lurk, 

from what it does among the most and develops he isting the esenderngs of the spection of history, and still of the serect and deckselves to the strancess of the spection of
commanned the reason and still ofference of the intime or as a serficity of the serevening to the livess of the surerficis of the seds and still offering to an
all accorments and should do a sernergions of the esender and still offering to an
all

----- diversity: 0.5
----- Generating with seed: "from what it does am"
from what it does among a same a some the symparh and these finding the and
commanine the sivity, and amon a serional ass deepress to freat as to a man ating they and self and then it is as advandering therebelt and precalle of the like the some
phingly and the spection of
itselficity of present that is to the inti has as his
seare to deeming to an it is the chreid of the interals, the contial continct opition of the

----- diversity: 1.0
----- Generating with seed: "from what it does am"
from what it

but is the straniments, the inditions and morals, the most and still of the strain to the indition, and not the most and stand into sentious of the philosophers and the most and morals, the experien and endousle of the most and stand the straniments, the world to dective, with the inapare of the most the individual of the most and stand intrealt, present more self-sayen and morals, the experien on

----- diversity: 0.5
----- Generating with seed: "an be laid hold of.
"
an be laid hold of.

hen itself that it is and still more hament of the spell of such a perfain to experien and still of the entimate, or it is as a see the means of law as a surned the madical. it the madicalled to the unford eevinsondeness to itself to the morality of the reld of the eassable and still in the morality of the self, and dellibe or the porden and formerly and possed a sureen to and dellicament
and pro

----- diversity: 1.0
----- Generating with seed: "an be laid hold of.
"
an be laid hold of.
but so man, 

In [17]:
start_index = random.randint(0, len(text) - maxlen - 1)

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(100):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.
            
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
    sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: " too easily frighten"
 too easily frighten by the such any manifest of the present be and so be a curding, and the same in the such any made n

----- diversity: 0.5
----- Generating with seed: " too easily frighten"
 too easily frighten perhaps a surmity, as the socreess, the presenting the secrenturical
us which has not the conses of

----- diversity: 1.0
----- Generating with seed: " too easily frighten"
 too easily frightenness, and
ifacts of a meching and elevating. whether ne, and pain casterts to eply there is desirse 

----- diversity: 1.2
----- Generating with seed: " too easily frighten"
 too easily frightent., posse2redced therent of fevilings hiss. and armently timeligrding masitable prepartivesiby speen
